In [59]:
from src.controller.HwpController import HwpController
from collections import OrderedDict

import re, uuid
import numpy as np

In [87]:
import re, uuid
import numpy as np

from src.service.ParseService import ParseService


class ProperParser:
    def extract_columns(self, table_list):
        """
        한글 표에서 공통적인 컬럼 부분을 추출합니다.
        컬럼은 대부분 표의 시작 부분에 작성되기 때문에 row 값은 0 혹은 1에 위치하게 됩니다.
        이후 컬럼 값들이 딕셔너리로 저장되어 있기 때문에 중복을 제거 후 리스트로 반환합니다.
        """
        columns = []
        for items in table_list:
            for item in items:
                if item not in columns and int(item['row']) == 0 or int(item['row']) == 1:
                    columns.append(item)

        return columns
    
    def extract_non_column_data(self, table_list, columns):
        """
        한글 표에서 컬럼 부분을 제거한 나머지 데이터들을 반환합니다.
        """
        return [[item for item in items[len(columns):] if not int(item['colspan']) > 1]for items in table_list]
    
    def group_by_date(self, dict_list):
        """
        한글 표 데이터를 날짜 별로 분류하여 리스트로 저장하여 반환합니다.
        """
        group_list = []

        for items in dict_list:
            rows = list(set([int(item['row']) for item in items]))
            for row in rows:
                temp = [item for item in items if int(item['row'])==row]
                group_list.append(temp)
        
        return group_list
    
    def update_merge_data(self, group_list):
        """
        한글 표에 병합 처리된 셀에 대한 데이터 처리를 완료한 뒤 리스트로 반환합니다.
        병합 처리된 셀은 rowspan 값을 이용하여 탐지한 뒤 해당 row값이 없는 리스트에 추가해 줍니다.
        """
        cached_merge_data = []
        cached_head_data = []
        
        for idx, items in enumerate(group_list):
            head_data = [data for data in items if int(data['col']) == 0 and int(data['rowspan']) > 1]
            merge_data = [data for data in items if int(data['col']) != 0 and int(data['rowspan']) > 1]
            
            if head_data:
                cached_head_data = head_data
            if merge_data:
                cached_merge_data = merge_data

            new_items = []
            if any(data not in items for data in cached_head_data):
                new_items.extend(cached_head_data)
            if any(data not in items for data in cached_merge_data):
                new_items.extend(cached_merge_data)

            group_list[idx] = new_items + items
        
        return group_list
    
    def serialize_to_dict(self, group_list, columns):
        """
        컬럼 리스트와 파싱이 끝난 그룹 리스트를 이용해서 데이터를 분류한 뒤 리스트로 반환합니다.
        컬럼 리스트에 대응하는 값들을 그룹 리스트에서 찾아서 추가해주는 작업을 수행합니다.
        """
        serialize_list = []

        for items in group_list:
            # data = {column: item['text'] for column, item in zip(columns, items) if column != '발파횟수' and int(item['colspan']) <= 1}
            data = {}
            for column, item in zip(columns, items):
                print(column, item)
                # serialize_list.append(data)

        # return serialize_list

In [88]:
service = ProperParser()
controller = HwpController()
path = '/Users/dobby/Desktop/Workspace/2월 계측보고서(호남고철5-2공구).hwp'

In [76]:
controller.get_table_list(path, '일자별 계측 현황')
table_list = controller.get_list()

unmatched field end


In [92]:
columns = service.extract_columns(table_list)

columns

[{'row': '0', 'col': '0', 'colspan': '1', 'rowspan': '2', 'text': '일자'},
 {'row': '0', 'col': '1', 'colspan': '1', 'rowspan': '2', 'text': '발파횟수'},
 {'row': '0', 'col': '2', 'colspan': '1', 'rowspan': '2', 'text': '시간'},
 {'row': '0', 'col': '3', 'colspan': '3', 'rowspan': '1', 'text': '관리기준'},
 {'row': '0', 'col': '6', 'colspan': '3', 'rowspan': '1', 'text': '측정결과'},
 {'row': '0', 'col': '9', 'colspan': '1', 'rowspan': '2', 'text': '계측위치'},
 {'row': '1',
  'col': '3',
  'colspan': '1',
  'rowspan': '1',
  'text': '진동속도(cm/s)'},
 {'row': '1',
  'col': '4',
  'colspan': '1',
  'rowspan': '1',
  'text': '진동레벨(dB(V))'},
 {'row': '1',
  'col': '5',
  'colspan': '1',
  'rowspan': '1',
  'text': '소음레벨(dB(A))'},
 {'row': '1',
  'col': '6',
  'colspan': '1',
  'rowspan': '1',
  'text': '진동속도(cm/s)'},
 {'row': '1',
  'col': '7',
  'colspan': '1',
  'rowspan': '1',
  'text': '진동레벨(dB(V))'},
 {'row': '1',
  'col': '8',
  'colspan': '1',
  'rowspan': '1',
  'text': '소음레벨(dB(A))'},
 {'row': '1',
  

In [93]:
dict_list = service.extract_non_column_data(table_list, columns)

dict_list

[[{'row': '5', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '5', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '강정1리마을회관'},
  {'row': '6', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '2회'},
  {'row': '6', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '13:10'},
  {'row': '6', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '0.2'},
  {'row': '6', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '75'},
  {'row': '6', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '70'},
  {'row': '6', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '6', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '6', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '6', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '강정1리마을회관'},
  {'row': '7', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '3회'},
  {'row': '7', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '19:39'},
  {'row': '

In [94]:
group_list = service.group_by_date(dict_list)

group_list

[[{'row': '5', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '5',
   'col': '9',
   'colspan': '1',
   'rowspan': '1',
   'text': '강정1리마을회관'}],
 [{'row': '6', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '2회'},
  {'row': '6', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '13:10'},
  {'row': '6', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '0.2'},
  {'row': '6', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '75'},
  {'row': '6', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '70'},
  {'row': '6', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '6', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '6', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '6',
   'col': '9',
   'colspan': '1',
   'rowspan': '1',
   'text': '강정1리마을회관'}],
 [{'row': '7', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '3회'},
  {'row': '7', 'col': '2', 'colspan': '1', 'rowspan': '1', 'te

In [96]:
group_list = service.update_merge_data(group_list)

group_list

[[{'row': '5', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '5',
   'col': '9',
   'colspan': '1',
   'rowspan': '1',
   'text': '강정1리마을회관'}],
 [{'row': '6', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '2회'},
  {'row': '6', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '13:10'},
  {'row': '6', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '0.2'},
  {'row': '6', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '75'},
  {'row': '6', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '70'},
  {'row': '6', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '6', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '6', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'},
  {'row': '6',
   'col': '9',
   'colspan': '1',
   'rowspan': '1',
   'text': '강정1리마을회관'}],
 [{'row': '7', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '3회'},
  {'row': '7', 'col': '2', 'colspan': '1', 'rowspan': '1', 'te

In [28]:
service.serialize_to_dict(group_list, columns)


일자 {'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '3', 'text': '2월1일'}
발파횟수 {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '1회'}
시간 {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '7:04'}
관리기준 {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '0.2'}
측정결과 {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '75'}
계측위치 {'row': '2', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '70'}
진동속도(cm/s) {'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'}
진동레벨(dB(V)) {'row': '2', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'}
소음레벨(dB(A)) {'row': '2', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '감지안됨'}
진동속도(cm/s) {'row': '2', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '강정1리마을회관'}
일자 {'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '3', 'text': '2월1일'}
발파횟수 {'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '2회'}
시간 {'row': '3', 'col': '2', 'colspan': '1', 'rows

In [32]:
import pandas as pd

# pandas DataFrame으로 변환
df = pd.DataFrame(serialize_list)

# 계측 위치별로 데이터 프레임 분류
locations = df['계측위치'].unique()  # 계측 위치별 고유값을 찾음
print(df['계측위치'].unique())

# 각 위치별 데이터 프레임을 딕셔너리에 저장
dfs = {}
for location in locations:
    dfs[location] = df[df['계측위치'] == location]

# 결과 확인
# 예시: 과천지식정보타운 위치의 데이터 프레임 출력
print(dfs['군포가압장'])

['과천지식정보타운' '군포가압장' '청동기유적지']
       일자   발파시간   계측위치 진동속도(cm/s) 진동레벨(dB(V))  소음레벨(dB(A))
1    3월2일   7:00  군포가압장      0.064           -        60.61
4    3월4일   7:00  군포가압장      0.063           -        60.01
7    3월4일  17:40  군포가압장      0.029           -        61.32
10   3월5일  17:00  군포가압장       0.03           -        67.91
13   3월6일   7:00  군포가압장      0.095           -  105.94(암소음)
16   3월6일  16:50  군포가압장      0.038           -        52.03
19   3월7일   7:00  군포가압장      0.016           -         61.8
22   3월7일  14:35  군포가압장      0.025           -        67.27
25   3월8일   7:00  군포가압장      0.105           -   84.44(암소음)
28   3월8일  17:00  군포가압장      0.056           -        65.22
31   3월9일   7:00  군포가압장      0.081           -        66.01
34  3월11일   7:00  군포가압장      0.053           -        65.54
37  3월11일  16:50  군포가압장      0.089           -        61.03
40  3월12일   7:17  군포가압장      0.045           -        63.06
43  3월12일  14:40  군포가압장      0.023           -        69.58
46  3월13일 